In [2]:
from transformers import BertModel, BertTokenizer, XLNetModel, XLNetTokenizer
# from nltk.corpus import stopwords
# from nltk import PorterStemmer, WordNetLemmatizer
import pandas as pd
import numpy as np
import pickle
import re
from sklearn.model_selection import train_test_split
import stopwordsiso
from stopwordsiso import stopwords
# stopwords(["zh"])  # Chinese
import jieba
from sklearn.preprocessing import MultiLabelBinarizer
import tensorflow_hub as hub
import torch

In [2]:
# clean text and seg
stop = stopwords(["zh"])
def preprocessingTextFull(text, stop=stop, sep = ' '):
    text = text.lower() #text to lowercase
    text = re.sub(r'&lt;', '', text) #remove '&lt;' tag
    text = re.sub(r'<.*?>', '', text) #remove html
    text = re.sub(r'[0-9]+', '', text) #remove number
    text = " ".join([word for word in text.split() if word not in stop]) #remove stopwords
    text = re.sub(r'[^\w\s]', '', text) #remove punctiation
#   text = re.sub(r'[^\x00-\x7f]', '', text) #remove non ASCII strings
    for c in ['\r', '\n', '\t'] :
        text = re.sub(c, ' ', text) #replace newline and tab with tabs\
        text = re.sub('\s+', ' ', text) #replace multiple spaces with one space
#         text = ' '.join([lemmatizer.lemmatize(word) for word in text.split()])
    text_cut = sep.join(jieba.cut(text, cut_all=False))
    
    return text_cut

## 3 Universal Sentence Encoder

In [3]:
df = pd.read_csv('training_sim.csv')
df = df.rename(columns={'sim_label': 'label','sim_detail_label': 'detail_label','sim_text': 'text' }, index={'ONE': 'Row_1'})
df = df.dropna()
df.head(3)

,label,detail_label,text
0,行销活动问题,未收到活动回馈金问题,"您好,我于2020/7/6 下午5:38左右绑定兆丰银行并开启自动储值功能,但仍未收到88元..."
1,行销活动问题,未收到活动点数问题,刷屈臣氏480元，却无点数回馈?
2,行销活动问题,未收到活动点数问题,已经确认过，你们只有给4月跟6月喔\n\n5月的点数是漏发的，5月帐单缴费13075元\n\...


In [4]:
df['text'] = df['text'].apply(preprocessingTextFull)
df['label'] = df['label'].apply(preprocessingTextFull)
df['detail_label'] = df['detail_label'].apply(preprocessingTextFull)
df.head(3)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\dupho\AppData\Local\Temp\jieba.cache
Loading model cost 0.621 seconds.
Prefix dict has been built successfully.


,label,detail_label,text
0,行销 活动 问题,未 收到 活动 回馈 金 问题,您好 我于 下午 左右 绑定 兆丰 银行 并 开启 自动 储值 功能 但 仍 未 收到 ...
1,行销 活动 问题,未 收到 活动 点数 问题,刷 屈臣氏 元 却 无 点数 回馈
2,行销 活动 问题,未 收到 活动 点数 问题,已经 确认 过 你们 只有 给 月 跟 月 喔 月 的 点数 是 漏发 的 月 帐单 缴...


In [5]:
# Generating Vectors using the Universal Sentence Encoder 
embed = hub.load("https://tfhub.dev/google/nnlm-zh-dim128/2")

INFO:absl:Using C:\Users\dupho\AppData\Local\Temp\tfhub_modules to cache modules.
INFO:absl:Downloading TF-Hub Module 'https://tfhub.dev/google/nnlm-zh-dim128/2'.
INFO:absl:Downloading https://tfhub.dev/google/nnlm-zh-dim128/2: 20.00MB
INFO:absl:Downloading https://tfhub.dev/google/nnlm-zh-dim128/2: 40.00MB
INFO:absl:Downloading https://tfhub.dev/google/nnlm-zh-dim128/2: 60.00MB
INFO:absl:Downloading https://tfhub.dev/google/nnlm-zh-dim128/2: 80.00MB
INFO:absl:Downloading https://tfhub.dev/google/nnlm-zh-dim128/2: 100.00MB
INFO:absl:Downloading https://tfhub.dev/google/nnlm-zh-dim128/2: 120.00MB
INFO:absl:Downloading https://tfhub.dev/google/nnlm-zh-dim128/2: 140.00MB
INFO:absl:Downloading https://tfhub.dev/google/nnlm-zh-dim128/2: 160.00MB
INFO:absl:Downloading https://tfhub.dev/google/nnlm-zh-dim128/2: 180.00MB
INFO:absl:Downloading https://tfhub.dev/google/nnlm-zh-dim128/2: 200.00MB
INFO:absl:Downloading https://tfhub.dev/google/nnlm-zh-dim128/2: 220.00MB
INFO:absl:Downloading https

In [6]:
#generate embeddings
textEmbeddings = embed(df['text'])#create list from np arrays
text_vec = np.array(textEmbeddings).tolist()#add lists as dataframe column
df['text_vec'] = text_vec #check dataframe df['use'] = [v for v in use]

labelEmbeddings = embed(df['label'])#create list from np arrays
label_vec = np.array(labelEmbeddings).tolist()#add lists as dataframe column
df['label_vec'] = label_vec #check dataframe df['use'] = [v for v in use]

df.head(2)


,label,detail_label,text,text_vec,label_vec
0,行销 活动 问题,未 收到 活动 回馈 金 问题,您好 我于 下午 左右 绑定 兆丰 银行 并 开启 自动 储值 功能 但 仍 未 收到 ...,"[-0.6959822773933411, -0.12320687621831894, -0...","[-0.19086284935474396, -0.0249605905264616, -0..."
1,行销 活动 问题,未 收到 活动 点数 问题,刷 屈臣氏 元 却 无 点数 回馈,"[-0.13868018984794617, -0.048432230949401855, ...","[-0.19086284935474396, -0.0249605905264616, -0..."


In [7]:
df[['text_vec','label_vec']].to_csv('train_vec.csv')
# ==============================================================================================================================================================================================================